# Day 1 - Trebuchet

The newly-improved calibration document consists of lines of text; each line originally contained a specific calibration value that the Elves now need to recover. On each line, the calibration value can be found by combining the first digit and the last digit (in that order) to form a single two-digit number.

For example:
```
1abc2
pqr3stu8vwx
a1b2c3d4e5f
treb7uchet
```
In this example, the calibration values of these four lines are 12, 38, 15, and 77. Adding these together produces 142.

Consider your entire calibration document. What is the sum of all of the calibration values?

In [ ]:
# Day 1, challenge 1

def calsum(filename):
    calvals = []
    with open(filename, 'r') as file:
        caldoc = file.read()
        first_digit = -1
        second_digit = -1

        #I'm going to just iterate through the whole doc char by char
        # I'm really only looking for two cases, digits and newlines
        for char in caldoc:
            # if it's a digit store it
            if char.isdigit():
                #Just keep updating the second digit every time you see a new one
                second_digit = int(char)
                #Only update the first digit the first time we see one
                if first_digit < 0:
                    first_digit = int(char)
            #if there's a new line or end of file, store the digit for this line and reset
            elif char == '\n' or char == '':
                # First save the calibration number
                calvals.append(first_digit * 10 + second_digit)
                # reset the vars for the digits
                first_digit = -1
                second_digit = -1

        # handle the case if the last line doesn't have a newline or EoF
        if first_digit >= 0 and second_digit >= 0:
            calvals.append(first_digit * 10 + second_digit)
            
    return sum(calvals)

calsum('AoC23-1CalibrationDoc.txt')

# Day 1 - Challenge 2

Your calculation isn't quite right. It looks like some of the digits are actually spelled out with letters: one, two, three, four, five, six, seven, eight, and nine also count as valid "digits".

Equipped with this new information, you now need to find the real first and last digit on each line. For example:
```
two1nine
eightwothree
abcone2threexyz
xtwone3four
4nineeightseven2
zoneight234
7pqrstsixteen
```
In this example, the calibration values are 29, 83, 13, 24, 42, 14, and 76. Adding these together produces 281.

What is the sum of all of the calibration values?


In [ ]:
# Day 1, challenge 2

def calsum2(filename):
    calvals = []
    numbers = ['one','two','three','four','five','six','seven','eight','nine']
    with open(filename, 'r') as file:
        caldoc = file.read()
        first_digit = -1
        second_digit = -1
        doclength = len(caldoc)

        #I'm going to just iterate through the whole doc char by char
        # I'm really only looking for two cases, digits and newlines
        for i,char in enumerate(caldoc):
            if char.isdigit():
                second_digit = int(char)
                if first_digit < 0:
                    first_digit = int(char)
            elif char == '\n' or char == '':
                # First save the calibration number
                calvals.append(first_digit * 10 + second_digit)
                first_digit = -1
                second_digit = -1
            else:
                for n,number in enumerate(numbers):
                    if len(number) + i > doclength:
                        next
                    elif caldoc[i:i+len(number)] == number:
                        second_digit = n + 1
                        if first_digit < 0:
                            first_digit = n + 1
                            

        # handle the case if the last line somehow doesn't have a newline or EoF
        if first_digit >= 0 and second_digit >= 0:
            calvals.append(first_digit * 10 + second_digit)
            
    return sum(calvals)

calsum2('AoC23-1CalibrationDoc.txt')

# Day 2 - Game 1

As you walk, the Elf shows you a small bag and some cubes which are either red, green, or blue. Each time you play this game, he will hide a secret number of cubes of each color in the bag, and your goal is to figure out information about the number of cubes.

To get information, once a bag has been loaded with cubes, the Elf will reach into the bag, grab a handful of random cubes, show them to you, and then put them back in the bag. He'll do this a few times per game.

You play several games and record the information from each game (your puzzle input). Each game is listed with its ID number (like the 11 in Game 11: ...) followed by a semicolon-separated list of subsets of cubes that were revealed from the bag (like 3 red, 5 green, 4 blue).

For example, the record of a few games might look like this:
```
Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green
```
In game 1, three sets of cubes are revealed from the bag (and then put back again). The first set is 3 blue cubes and 4 red cubes; the second set is 1 red cube, 2 green cubes, and 6 blue cubes; the third set is only 2 green cubes.

The Elf would first like to know which games would have been possible if the bag contained only 12 red cubes, 13 green cubes, and 14 blue cubes?

In the example above, games 1, 2, and 5 would have been possible if the bag had been loaded with that configuration. However, game 3 would have been impossible because at one point the Elf showed you 20 red cubes at once; similarly, game 4 would also have been impossible because the Elf showed you 15 blue cubes at once. If you add up the IDs of the games that would have been possible, you get 8.

Determine which games would have been possible if the bag had been loaded with only 12 red cubes, 13 green cubes, and 14 blue cubes. What is the sum of the IDs of those games?

In [ ]:
# Day 2, challenge 1
import re

# Inputs:
# filename - The game file to load
# noisy - Whether to print out a bunch of stuff while running
# target - A dict with keys based on colors and values basec on cube counts that we're testing against
# Returns: sum of game ids that are valid
def cubegame(filename,noisy=True,target={'red':12,'blue':14,'green':13}):
    # this will hold the sum of game ids for those games that were possible
    possible_sum = 0
    # This will tally the maximum number of each color seen in the grabs for a game. It's reset for each game
    game_tally = {'red':0,'blue':0,'green':0}
    
    # re for matching game id
    # Slightly brittle, assumes Game is capitalized
    gamere = re.compile(r'Game\s+(\d+)\:')
    #re for matching the colors in each grab.
    # Slightly brittle, assumes number comes first and nothing is capitalized
    grabre = re.compile(r'(\d+)\s+(red|blue|green)')
    
    with open(filename, 'r') as file:
        # Read line by line
        for line in file:
            # First Grab the Game ID
            game_index = gamere.match(line).group(1)
            
            # Then re match each handful and iterate through the grabs
            # We don't need to separate by the individual grabs (separated by semicolons), because 
            # ... we assume each color appears only once per grab
            for count,color in grabre.findall(line):
                game_tally[color] = max(int(count),int(game_tally[color]))
                
            print(f"Game {game_index}: red {game_tally['red']}, blue {game_tally['blue']}, green {game_tally['green']}") if noisy else None           

            # if the maximum number we saw across the grabs is not greater than the target, we know that target is possible
            if game_tally['red'] <= target['red'] and game_tally['blue'] <= target['blue'] and game_tally['green'] <= target['green']:
                possible_sum += int(game_index)
                print(f"Sum = {possible_sum}") if noisy else None
            game_tally = {'red':'0','blue':'0','green':'0'}
            
    return possible_sum

cubegame('AoC23-2Game.txt',noisy=False)

# Day 2, Game 2

As you continue your walk, the Elf poses a second question: in each game you played, what is the fewest number of cubes of each color that could have been in the bag to make the game possible?

Again consider the example games from earlier:
```
Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green
```
    In game 1, the game could have been played with as few as 4 red, 2 green, and 6 blue cubes. If any color had even one fewer cube, the game would have been impossible.
    Game 2 could have been played with a minimum of 1 red, 3 green, and 4 blue cubes.
    Game 3 must have been played with at least 20 red, 13 green, and 6 blue cubes.
    Game 4 required at least 14 red, 3 green, and 15 blue cubes.
    Game 5 needed no fewer than 6 red, 3 green, and 2 blue cubes in the bag.

The power of a set of cubes is equal to the numbers of red, green, and blue cubes multiplied together. The power of the minimum set of cubes in game 1 is 48. In games 2-5 it was 12, 1560, 630, and 36, respectively. Adding up these five powers produces the sum 2286.

For each game, find the minimum set of cubes that must have been present. What is the sum of the power of these sets?

In [ ]:
# Day 2, challenge 2
import re

# Inputs:
# filename - The game file to load
# noisy - Whether to print out a bunch of stuff while running
# target - A dict with keys based on colors (red,green,blue) and values based on cube counts that we're testing against
# Returns: sum of game ids that are valid, sum of power for all games
def cubegame2(filename,noisy=True,target={'red':12,'blue':14,'green':13}):
    # this will hold the sum of game ids for those games that were possible
    possible_sum = 0
    # This will hold the sum of the powers
    power_sum = 0
    # list of colors for iteration purposes.
    # Compared to challenge one version, this would enable us to easily expand to more colors
    colors = ['red','green','blue']
    # This will tally the maximum number of each color seen in the grabs for a game. It's reset for each game
    game_tally = {'red':0,'blue':0,'green':0}
    # This is the difference between the target and the game_tally
    game_diff = {'red':0,'blue':0,'green':0}
    
    #re for matching game id
    gamere = re.compile(r'Game\s+(\d+)\:')
    #re for matching each grab
    # To do: will need to modify this if we add colors
    grabre = re.compile(r'(\d+)\s+(red|blue|green)')
    
    with open(filename, 'r') as file:
        # Read line by line
        for line in file:
            # Reset things. tally to 0 and power to 1 because multiplication
            power = 1
            game_tally = {'red':'0','blue':'0','green':'0'}
            # First Grab the Game ID
            game_index = gamere.match(line).group(1)
            
            # Then re match each handful and iterate through the grabs
            # We don't need to separate by the individual grabs (separated by semicolons), because 
            # ... we assume each color appears only once per grab
            for count,color in grabre.findall(line):
                game_tally[color] = max(int(count),int(game_tally[color]))
                
            print(f"Game {game_index}: red {game_tally['red']}, blue {game_tally['blue']}, green {game_tally['green']}") if noisy else None
            
            # Generalized a bit from above. Instead of specifying red, green blue, iterate through list of colors
            for color in colors:                
                # The game tally is by definition the minimum you'd need for the game so just multiply each color
                power = power * game_tally[color]
                # calculate a diff from the target for testing valid games in challenge 1
                game_diff[color] = target[color] - game_tally[color]

            # Add power to the running sum
            power_sum += power
            print(f"Power = {power}, Power Sum = {power_sum}") if noisy else None
                
            if game_diff['red'] >= 0 and game_diff['blue'] >= 0 and game_diff['green'] >= 0:
                possible_sum += int(game_index)
                print(f"Sum = {possible_sum}") if noisy else None
            
    return possible_sum, power_sum

cubegame2('AoC23-2Game.txt',noisy=False)

# Day 3, Puzzle 1

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

Here is an example engine schematic:
```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```
In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

Of course, the actual engine schematic is much larger. What is the sum of all of the part numbers in the engine schematic?

In [ ]:
# Day 3, challenge 1

def engineparts(filename):
    partsum = 0 
    with open(filename, 'r') as file:

        docmatrix = [line for line in file]
        doclength = len(docmatrix)
        # Just assume all lines are the same length and read the first line here
        for lidx,ln in enumerate(docmatrix):
            ln = ln.rstrip()
            activenumber = -1
            # If I've found a symbol in the active char, the one above or below
            # This will get cleared next time around, unles I'm in a number
            prevsymbol = False
            for cidx,char in enumerate(ln):
                # First check if there's a new symbol here, one below or one above
                chardigit = char.isdigit()
                activesymbol = False
                for y in [-1,0,1]:
                    newidx = lidx+y
                    if newidx < 0 or newidx >= doclength-1:
                        next
                    elif docmatrix[newidx][cidx] != '.' and docmatrix[newidx][cidx].isdigit() == False:
                        activesymbol = True
                if activenumber >=0:
                    #This means I've already started reading in a number
                    #first check if this line is another number
                    if chardigit:
                        activenumber = activenumber * 10 + int(char)
                        prevsymbol = prevsymbol or activesymbol
                        if cidx == len(ln) - 1 and prevsymbol:
                            # We have a countable number here
                            partsum += activenumber
                            activenumber = -1
                            prevsymbol = activesymbol
                    elif activesymbol or prevsymbol:
                        # We have a countable number here
                        partsum += activenumber
                        activenumber = -1
                        prevsymbol = activesymbol
                    else:
                        activenumber = -1
                elif chardigit:
                    activenumber = int(char)
                    prevsymbol = prevsymbol or activesymbol
                else:
                    prevsymbol = activesymbol

        
    return partsum

engineparts('AoC23-3Engine.txt')

# Day 3, Challenge 2

The missing part wasn't the only issue - one of the gears in the engine is wrong. A gear is any * symbol that is adjacent to exactly two part numbers. Its gear ratio is the result of multiplying those two numbers together.

This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out which gear needs to be replaced.

Consider the same engine schematic again:

```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```

In this schematic, there are two gears. The first is in the top left; it has part numbers 467 and 35, so its gear ratio is 16345. The second gear is in the lower right; its gear ratio is 451490. (The * adjacent to 617 is not a gear because it is only adjacent to one part number.) Adding up all of the gear ratios produces 467835.

What is the sum of all of the gear ratios in your engine schematic?

In [ ]:
# Day 3, challenge 2

def combine_symbols(list1, list2):
    if list1 is None:
        return list2
    elif list2 is None:
        return list1
    else:
        return list1 + list2

def engineparts2(filename):
    partsum = 0
    
    # This is going to be a dictionary of all gears
    # The keys are the "id" of the gear, which is a string of the line index - character index
    # The value is a list of all part numbers connected to that gear
    gears = {}
    with open(filename, 'r') as file:

        docmatrix = [line for line in file]
        doclength = len(docmatrix)

        for lidx,ln in enumerate(docmatrix):
            ln = ln.rstrip()

            # This keeps of track of whether we're currently constructing a number
            activenumber = -1
            
            # This keeps track of whather we've seen a symbol before this
            # This either just keeps track of the previous activesymbol (see below) in case a number s starting
            # or if we're in the middle of a part number whether there have been any activesymbols since the beginning the number
            # It's None if there was no symbol in the previous char (or chars if tracking a number)
            # It's an empty list if there have been symbols, but no gears
            # It's a list of gear ids seen
            prevsymbol = None
            for cidx,char in enumerate(ln):
                # First check if there's a new symbol here, one below or one above
                chardigit = char.isdigit()

                # We're first trying to find a symbol in the active char, the one above, or the one below
                # This will get cleared next time around, unles I'm in a number
                # This is None if there are no symbols, an empty list if no gears, or a list of gear ids found
                activesymbol = None
                for y in [-1,0,1]:
                    newidx = lidx+y
                    # first make sure we're not going past the bounds of the data
                    if newidx < 0 or newidx >= doclength-1:
                        next
                    elif docmatrix[newidx][cidx] != '.' and docmatrix[newidx][cidx].isdigit() == False:
                        # We've now found a symbol (not a . or a digit)
                        activesymbol = []
                        if docmatrix[newidx][cidx] == '*':
                            # we have a gear here add the id to the activesymbol
                            gear_index = f'{newidx}-{cidx}'
                            activesymbol.append(gear_index)
                            # also add this key to the global gear dict
                            if gear_index not in gears:
                                gears[gear_index] = []
                if activenumber >=0:
                    #This means I've already started reading in a number
                    #first check if this line is another number
                    if chardigit:
                        #another digit, so keep adding to the active number
                        activenumber = activenumber * 10 + int(char)
                        #since we're mid number, combine symbols in previous and current
                        prevsymbol = combine_symbols(prevsymbol,activesymbol)
                        if cidx == len(ln) - 1 and prevsymbol != None:
                            # We have a countable number here and we've reached the end of the line
                            # Add it to the global part tally
                            partsum += activenumber
                            # Append this number to the gears this number has touched
                            for gidx in prevsymbol:
                                gears[gidx].append(activenumber)
                            #now clear things out
                            activenumber = -1
                            prevsymbol = None

                    # Not another number, so we've reached the end of a number
                    # if we've seen a symbol here or previously store the number
                    elif activesymbol != None or prevsymbol != None:
                        # We have a countable number here
                        partsum += activenumber
                        #even though I'm about to clear out the symbols I need all that this number 
                        # touched so I can append any gears to the global dict
                        prevsymbol = combine_symbols(prevsymbol,activesymbol)
                        for gidx in prevsymbol:
                            gears[gidx].append(activenumber)
                        activenumber = -1
                        # This is to cover the scenario where I've got a single space between two numbers
                        # I don't want keep all of the previous symbols, just the current one
                        # in case a new number starts on the next character
                        prevsymbol = activesymbol
                    else:
                        # not a number and symbols are None
                        # This means a number just ended, but there were no symbols
                        # symbols are already None, so just clear out the active number
                        activenumber = -1
                        
                # we aren't in the middle of a number, so this is the start of a part number
                elif chardigit:
                    activenumber = int(char)
                    prevsymbol = combine_symbols(prevsymbol,activesymbol)
                # Not in the middle of a number or starting a new one
                # Just hold onto symbols at this position in case a number starts next
                else:
                    prevsymbol = activesymbol

        
    #finally calulate gear ratios
    gearsum = 0
    for gidx in gears.keys():
        if len(gears[gidx]) == 2:
            gearsum += gears[gidx][0] * gears[gidx][1]
    
    return partsum, gearsum

engineparts2('AoC23-3Engine.txt')

# Day 4, Challenge 1

The Elf leads you over to the pile of colorful cards. There, you discover dozens of scratchcards, all with their opaque covering already scratched off. Picking one up, it looks like each card has two lists of numbers separated by a vertical bar (|): a list of winning numbers and then a list of numbers you have. You organize the information into a table (your puzzle input).

As far as the Elf has been able to figure out, you have to figure out which of the numbers you have appear in the list of winning numbers. The first match makes the card worth one point and each match after the first doubles the point value of that card.

For example:
```
Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11
```
In the above example, card 1 has five winning numbers (41, 48, 83, 86, and 17) and eight numbers you have (83, 86, 6, 31, 17, 9, 48, and 53). Of the numbers you have, four of them (48, 83, 17, and 86) are winning numbers! That means card 1 is worth 8 points (1 for the first match, then doubled three times for each of the three matches after the first).

    Card 2 has two winning numbers (32 and 61), so it is worth 2 points.
    Card 3 has two winning numbers (1 and 21), so it is worth 2 points.
    Card 4 has one winning number (84), so it is worth 1 point.
    Card 5 has no winning numbers, so it is worth no points.
    Card 6 has no winning numbers, so it is worth no points.

So, in this example, the Elf's pile of scratchcards is worth 13 points.

Take a seat in the large pile of colorful cards. How many points are they worth in total?

In [ ]:
# Day 4 Challenge 1
import re

def parse_line(line):
    num_winning_numbers = 5
    
    # Pretty simple lines to parse: Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
    # Grab a digit after Card, all digits until the pipe, then all after
    gamere = re.compile(r'Card\s+(\d+)\:\s*((\s|\d)+?)\s*\|\s+((\s|\d)+?)\s*$')
    result = gamere.match(line)
    
    game_no = result.group(1)
    # split out the single strings of numbers into individual numbers
    winning_numbers = re.split(r'\s+',result.group(2))
    my_numbers = re.split(r'\s+',result.group(4))

    return int(game_no), winning_numbers, my_numbers

# Calculate the score based on the rules
# Score is basically 2 to the power of the number of matches cards - 1
def lotto_score(winning_numbers = [], my_numbers = []):
    matches = 0
    for x in my_numbers:
        matches = matches + 1 if x in winning_numbers else matches
        
    
    return 2**(matches - 1) if matches > 0 else 0

def lotto(filename):

    points = 0
    with open(filename, 'r') as file:
        for line in file:
            gn, wn, mn = parse_line(line)
            points += lotto_score(wn,mn)
            
    return points

        
lotto('AoC23-4lotto.txt')

# Day 4, Challenge 2

Just as you're about to report your findings to the Elf, one of you realizes that the rules have actually been printed on the back of every card this whole time.

There's no such thing as "points". Instead, scratchcards only cause you to win more scratchcards equal to the number of winning numbers you have.

Specifically, you win copies of the scratchcards below the winning card equal to the number of matches. So, if card 10 were to have 5 matching numbers, you would win one copy each of cards 11, 12, 13, 14, and 15.

Copies of scratchcards are scored like normal scratchcards and have the same card number as the card they copied. So, if you win a copy of card 10 and it has 5 matching numbers, it would then win a copy of the same cards that the original card 10 won: cards 11, 12, 13, 14, and 15. This process repeats until none of the copies cause you to win any more cards. (Cards will never make you copy a card past the end of the table.)

This time, the above example goes differently:
```
Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11
```
    Card 1 has four matching numbers, so you win one copy each of the next four cards: cards 2, 3, 4, and 5.
    Your original card 2 has two matching numbers, so you win one copy each of cards 3 and 4.
    Your copy of card 2 also wins one copy each of cards 3 and 4.
    Your four instances of card 3 (one original and three copies) have two matching numbers, so you win four copies each of cards 4 and 5.
    Your eight instances of card 4 (one original and seven copies) have one matching number, so you win eight copies of card 5.
    Your fourteen instances of card 5 (one original and thirteen copies) have no matching numbers and win no more cards.
    Your one instance of card 6 (one original) has no matching numbers and wins no more cards.

Once all of the originals and copies have been processed, you end up with 1 instance of card 1, 2 instances of card 2, 4 instances of card 3, 8 instances of card 4, 14 instances of card 5, and 1 instance of card 6. In total, this example pile of scratchcards causes you to ultimately have 30 scratchcards!

Process all of the original and copied scratchcards until no more scratchcards are won. Including the original set of scratchcards, how many total scratchcards do you end up with?

In [ ]:
# Day 4 Challenge 2
import re

# Simply return the number of matched numbers in the winning numbers
def lotto_matches(winning_numbers = [], my_numbers = []):
    matches = 0
    for x in my_numbers:
        matches = matches + 1 if x in winning_numbers else matches
            
    return matches

def lotto2(filename):

    # Keep a dict of card numbers
    # keys are the card numbers
    # values are the count for that card given the rules
    cards = {}
    # Keep track of the last card number we've seen to ensure we don't go past it
    maxgn = 0
    with open(filename, 'r') as file:
        for line in file:
            gn, wn, mn = parse_line(line)
            matches = lotto_matches(wn,mn)
            maxgn = max(gn,maxgn)
            # add one card for the original card
            cards[gn] = 1 + cards[gn] if gn in cards.keys() else 1
            # add matches number of cards for the matches
            for i in range(gn+1,gn+matches+1):
                cards[i] = cards[gn] + cards[i] if i in cards.keys() else cards[gn] 
    
    # Now clear out any cards we created past the max game
    # At most you can have ten matches, so go five past the max
    for i in range(maxgn+1,maxgn+11):
        if i in cards.keys():
            cards[i] = 0
    return sum(cards.values())

        
lotto2('AoC23-4lotto.txt')